# Subindo dados da camada Bronze 

## Importando dados e gerando conexão com o postgres

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import os

# 2. Configurar a conexão com o banco
usuario = "meu_usuario"
senha = "minha_senha_segura"
host = "localhost"
porta = "5432"
banco = "meu_banco"

db_url = f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco}"
engine = create_engine(db_url)

### Lendo dataframe olist_orders com a query

In [2]:
nome_tabela = 'olist_orders'

# Crie a query SQL qualificando a tabela com o schema
query = f"SELECT * FROM bronze.{nome_tabela};"

# Execute a query
# O uso do 'text()' é uma boa prática para evitar problemas de formatação com SQLAlchemy
df_query = pd.read_sql_query(text(query), engine)

# 3. Visualizar os primeiros registros do DataFrame
print("Tabela lida com sucesso com a query!")

Tabela lida com sucesso com a query!


In [3]:
df_query.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [4]:
df_query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


### Alterando type correto da coluna order_purchase_timestamp

In [5]:
df_query["order_purchase_timestamp"] = pd.to_datetime(df_query["order_purchase_timestamp"])

### Verificando valores nulos

In [6]:
df_query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  object        
 5   order_delivered_carrier_date   97658 non-null  object        
 6   order_delivered_customer_date  96476 non-null  object        
 7   order_estimated_delivery_date  99441 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 6.1+ MB


### Alterando o formato datetime para melhor visualização da coluna

In [7]:
df_query["order_estimated_delivery_date"] = pd.to_datetime(df_query["order_estimated_delivery_date"], errors='coerce').dt.strftime("%d/%m/%Y")
df_query["order_estimated_delivery_date"] = pd.to_datetime(df_query["order_estimated_delivery_date"])

C:\Users\Legion\AppData\Local\Temp\ipykernel_26660\2775767977.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_query["order_estimated_delivery_date"] = pd.to_datetime(df_query["order_estimated_delivery_date"])


In [8]:
df_query[['order_estimated_delivery_date']]

,order_estimated_delivery_date
0,2017-10-18
1,2018-08-13
2,2018-09-04
3,2017-12-15
4,2018-02-26
...,...
99436,2017-03-28
99437,2018-03-02
99438,2017-09-27
99439,2018-02-15


In [9]:
df_query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  object        
 5   order_delivered_carrier_date   97658 non-null  object        
 6   order_delivered_customer_date  96476 non-null  object        
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(6)
memory usage: 6.1+ MB


In [10]:
df_query["order_approved_at"] = pd.to_datetime(df_query["order_approved_at"])

In [11]:
df_query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  object        
 6   order_delivered_customer_date  96476 non-null  object        
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](3), object(5)
memory usage: 6.1+ MB


In [12]:
df_query["order_delivered_carrier_date"] = pd.to_datetime(df_query["order_delivered_carrier_date"])

In [13]:
df_query["order_delivered_customer_date"] = pd.to_datetime(df_query["order_delivered_customer_date"])

### Verificando alteração Datatime

In [14]:
df_query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


### Subindo para camada Silver

In [15]:
df_query.to_sql(
        "olist_orders",
        engine,
        schema="silver",  # <-- AQUI ESTÁ A MUDANÇA PRINCIPAL
        if_exists='replace',
        index=False
    )

print(f"Tabela olist_customers criada com sucesso no schema silver!")

Tabela olist_customers criada com sucesso no schema silver!
